# Exercise A2 (b)

After finishing his undergrad studies, your good friend *Dieter Schlau* took up a job as a sysadmin. Now he reaches out to you for help with a sequential decision-making problem that involves network maintenance. In this notebook you will mainly implement a simulator/reinforcement learning environment for his problem.

## README
We strongly recommend you install `gymnasium>=0.28.1` and `python>=3.10.*`. You can set up a python environment using e.g. [conda](https://docs.conda.io/projects/miniconda/en/latest/) in the terminal
```
conda create -n rl23 python=3.10 gymnasium=0.28.1 notebook -c conda-forge
conda activate rl23
```
The `notebook` package is required for opening and working with this jupyter notebook.
In a terminal with the Python environment active, run
```
jupyter notebook Notebook_A.ipynb
```

If you need help setting up a development environment, please visit the office hour or ask for help in the forum.

## Submission
**Solve task (I), (II), and (III) and submit your final `.ipynb` file as a solution to the mCMS.**

During grading, we will clear outputs and then run all cells.
Notebooks that produce runtime errors will be graded with 0 points for Task (II) and (III), unless the errors arise due to gymnasium/numpy backwards compatibility issues.

In [1]:
from typing import Any, Dict, Callable
from dataclasses import dataclass
from functools import cached_property, partial

import numpy as np
import gymnasium as gym
from gymnasium import spaces

## Problem statement
#### Network topology
The sequential decision making problem Dieter needs help solving, involves maintaining a network of $n > 1$ **servers (nodes)**,
connected through (undirected) edges denoted by $c_{ij} = c_{ji} = 1\{i \text{ and } j \text{ are connected}\}$.
These **fixed** connections arise from the fact the servers depend on each other to some extent.

In [3]:
@dataclass
class SysadminInstance:
    """
    A class that captures the concrete network instance, i.e.
    the number of servers (n), the connections (c_{ij})
    and some values for the parameters alpha and epsilon
    that are discussed below.
    """
    num_servers: int
    connections: list[tuple[int, int]]
    alpha: float
    eps: float
   
    @cached_property
    def adjacency(self) -> np.ndarray:
        """
        Connections c_{..} gathered in a matrix.
        Returns
        -------
        np.ndarray of shape (n,n)
        
        """
        adjacency = np.zeros((self.num_servers, self.num_servers), dtype=np.int8)
        for (i, j) in self.connections:
            adjacency[i, j] = 1
            adjacency[j, i] = 1
        return adjacency
    
RUNNING, CRASHED = 0, 1

*Note*: The particular network Dieter is currently maintaining is fixed/defined below

In [4]:
instance = SysadminInstance(
    num_servers=6,
    connections=[(0,1), (1,2), (1,3), (2,3), (2,4), (3,5)],
    alpha=0.5,
    eps=0.1
)

#### Network dynamics
Every day $t=0,1,2,...$, a server $i$ can be in one of two states
	$X_i^{(t)} \in \{0, 1\}$, 
`RUNNING` ($0$), or `CRASHED` ($1$).
Dieter's objective is to minimize
\begin{align*}
\sum_{t=0}^{\infty} \frac{\gamma^t}{n} \sum_{i=1}^n X_i^{(t)}, %& (\gamma \in [0,1])
\end{align*}
the cumulative, discounted fraction of `CRASHED` servers.
Right now ($t=0$), all servers are `RUNNING`.
On each day $t$, Dieter must decide which of the servers should be maintained (only one can be maintained a time, $A_t \in \{1,\ldots,n\}$)
such that its state on day $t+1$ is guaranteed to be `RUNNING`.
All other `RUNNING` servers may randomly crash, depending on the state of their neighbors.
Unmaintained servers that are already `CRASHED` remain `CRASHED`.
More precisely, for all $i$
\begin{align*}
P\left(X_i^{(0)} = 0\right) &= 1 \\
P\left(X_i^{(t+1)} = 0 \mid A_t = i\right) &= 1 \\
P\left(X_i^{(t+1)} = 1 \mid A_t \neq i, X^{(t)}_i = 0\right) &= \epsilon + \frac{\alpha C_i^{(t)}}{\sum_{j} c_{ij}} \\
P\left(X_i^{(t+1)} = 1 \mid A_t \neq i, X^{(t)}_i = 1\right) &= 1
\end{align*}
where $C_i^{(t)} = \sum_{j} c_{ij} X_i^{(t)}$ is the number of `CRASHED` neighbors of 
$i$ and $\epsilon > 0$, $\alpha > 0$, $\epsilon + \alpha \leq 1$
are additional parameters.

Assume that *changes* in state of any two servers $i \neq j$ occur independently.

## Task (I)
*Briefly* describe the above maintenance task as an MDP. Your description does not need to be mathematically rigorous.

**Answer (I):**
...

## Task (II)
Complete the implementation of the environment class `SysadminEnv` below (that acts as a model-free simulator for the above MDP) by implementing a [step function and a reset function](https://gymnasium.farama.org/api/env/).
- The `reset` function should reset the simulator and return the appropriate initial state.
- The `step` function should advance the simulator by one step,  i.e.,
  1. based on the current state and a given action, sample (from the correct distribution) a successor state
  3. return the successor state, the appropriate reward & additional information as specified by the gymnasium docs.

You may also extend other parts of the code.
Your code will not be graded if it produces errors when run, or if it contains syntax errors.

In [ ]:
class SysadminEnv(gym.Env):

    def __init__(
        self,
        instance: SysadminInstance,
    ) -> None:
        super().__init__()
        self.instance = instance
        self.action_space = spaces.Discrete(self.instance.num_servers)
        self.observation_space = spaces.MultiBinary(self.instance.num_servers)
        ... # extend this if needed

    def reset(
        self, *, seed: int | None = None, options: dict[str, Any] | None = None
    ) -> tuple[np.ndarray, dict[str, Any]]:
        super().reset(seed=seed)
        # TODO: implement this
        initial_state = ...
        ...
        return initial_state
   
    def step(self, action: int) -> tuple[np.ndarray, float, bool, bool, dict[str, Any]]:
        # TODO: implement this
        next_state = ...
        reward = ...
        ...
        return next_state, reward, False, False, dict()

In [ ]:
gym.register("Sysadmin-EA2", partial(SysadminEnv, instance))
env = gym.make("Sysadmin-EA2")
state, _ = env.reset()
state, reward, terminated, truncated, info = env.step(0)

### Task (III)
Dieter's current maintenance policy $\pi_D$ is given by:

In [ ]:
class RandomMaintenancePolicy:
    def __init__(self, seed: int | None = None) -> None:
        self.rng = np.random.default_rng(seed=seed)
    
    def __call__(self, state: np.ndarray) -> int:
        # maintain uniformly at random if all servers running
        if np.all(state == RUNNING):
            return self.rng.integers(0, state.shape[0])
        # randomly choose a crashed server for maintenance else
        pi = self.rng.random(state.shape[0])
        pi[state == RUNNING] = 0
        return np.argmax(pi)

policy_dieter = RandomMaintenancePolicy(seed=42)

Help Dieter to estimate $\mathbb{E}_{\pi_D}[G_0]$
Assume that the task is limited by a finite horizon, i.e. $t=0,\dots,T-1$ with $\gamma=1$ and $T=100$.

Proceed in the following way;
1. Implement the function `sample_return` below.
2. Call the function 100 times and compute the sample mean.

In [ ]:
def sample_return(
    env: SysadminEnv,
    policy: Callable[[np.ndarray], int],
    horizon: int = 100,
    gamma: float = 1,
    seed: int | None = 42,
) -> float:
    """
    Perform a fixed amount of steps using actions from a provided policy
    to generate a sample of the return "G_0" in a finite horizon/episodic setting
    by accumulating discounted rewards for a fixed number of steps and returning the result.

    
    Parameters
    ----------
    env : SysadminEnv
    policy : Callable[[np.ndarray], int]
    horizon : int, (T)
    gamma: float, discount factor
    seed: int | None, environment random seed
    
    Returns
    -------
    float
        A single sample of the return as described above. 
    """
    state, _ = env.reset(seed=seed)
    # TODO
    ...

In [ ]:
# TODO
value_estimate = ...